In [48]:
!nvidia-smi

Mon Apr 22 03:42:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB           Off| 00000000:07:00.0 Off |                    0 |
| N/A   66C    P0              410W / 400W|  80636MiB / 81920MiB |    100%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [30]:
import torch
import os

In [31]:
os.getcwd()

'/raid/tungks/gttm/reseach'

In [32]:
checkpoint = torch.load('../runs/detect/train5/weights/best.pt')
model = checkpoint['model'].float()

In [38]:
checkpoint.keys()

dict_keys(['epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'train_args', 'train_metrics', 'train_results', 'date', 'version', 'license', 'docs'])

In [43]:
checkpoint['ema']

In [15]:
from PIL import Image
import numpy as np

In [16]:
image = Image.open('/home/tungks/shared_drive_cv/GTTMData/external/traffic_camera/train/images/aguanambi-1000_png_jpg.rf.0ab6f274892b9b370e6441886b2d7b9d.jpg')
image = np.array(image) 
image = image / 255.0 
image = np.transpose(image, (2, 0, 1))  # Transpose to (channels, height, width)
image = np.expand_dims(image, axis=0)  # Add batch dimension
image = image.astype(np.float32)

input_tensor = torch.from_numpy(image)

with torch.no_grad():
    output = model(input_tensor)

print(output[0].shape)

torch.Size([1, 9, 8400])


# Inference

In [1]:
from ultralytics import YOLO

In [2]:
# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Define path to video file
source = '/raid/tungks/shared_drive_cv/GTTMData/external/youtube/videos/nga-tu-so_cutted.mp4'

In [3]:
# Run inference on the source
results = model(source, stream=True)
results

<generator object BasePredictor.stream_inference at 0x7fcf676eef90>

In [4]:
for r in results:
    print(r[0])
    break

/home/minhnh/python_venv/cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


video 1/1 (frame 1/12093) /raid/tungks/shared_drive_cv/GTTMData/external/youtube/videos/nga-tu-so_cutted.mp4: 384x640 13 persons, 15 cars, 5 motorcycles, 2 buss, 96.2ms
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47

In [3]:
from ultralytics import YOLO

In [4]:
model = YOLO('yolov8n.pt')

In [28]:
import cv2

video_path = '/raid/tungks/shared_drive_cv/GTTMData/external/youtube/videos/nga-tu-so_cutted.mp4'
cap = cv2.VideoCapture(video_path)
print('video capture')
i = 0

while cap.isOpened():
    if i > 20:
        break
    success, frame = cap.read()
   
    if success:
        print(f'frame {i} read succeeded')
        cv2.imwrite(f'my_video_frame_{i}.png', frame)
        print('predicting')
        results = model.predict(frame)

        tensor = results[0].boxes.data
        zeros_column = torch.zeros(tensor.shape[0], 1, device=tensor.device)
        zeros_column.fill_(i)
        new_tensor = torch.cat((zeros_column, tensor), dim=1)
        if i == 0:
            stacked_tensor = new_tensor
        else:
            stacked_tensor = torch.cat((stacked_tensor, new_tensor), dim=0)
        
        i += 1

print(stacked_tensor)

video capture
frame 0 read succeeded
predicting

0: 384x640 13 persons, 15 cars, 5 motorcycles, 2 buss, 5.5ms
Speed: 1.9ms preprocess, 5.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
frame 1 read succeeded
predicting

0: 384x640 13 persons, 15 cars, 3 motorcycles, 2 buss, 5.2ms
Speed: 1.0ms preprocess, 5.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
frame 2 read succeeded
predicting

0: 384x640 15 persons, 16 cars, 3 motorcycles, 2 buss, 5.1ms
Speed: 1.0ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
frame 3 read succeeded
predicting

0: 384x640 14 persons, 14 cars, 3 motorcycles, 2 buss, 5.3ms
Speed: 1.1ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
frame 4 read succeeded
predicting

0: 384x640 13 persons, 14 cars, 2 motorcycles, 2 buss, 5.3ms
Speed: 1.0ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
frame 5 read succeeded
pre

In [27]:
stacked_tensor.shape

torch.Size([199, 7])

# Test

In [ ]:
model = YOLO('yolov8n.pt')
results = model.train(data='coco8.yaml', epochs=100, imgsz=640)

In [1]:
import os

def check_file_suffix(file_path):
    # Get the file extension
    _, file_extension = os.path.splitext(file_path)

    # Check the file extension
    if file_extension == ".pt":
        print(".pt")
    elif file_extension in [".yaml", ".yml"]:
        print("yaml")
    else:
        print("Unknown suffix")

# Example usage
file_path = "usr/test/python/readme.yaml"
check_file_suffix(file_path)

yaml
